This program assigns roles to team members for the project. 

Conditions:
1. Team will rotate team roles in "cycles".
2. The length of each cycle may be two weeks or more, as determined by the team.
3. Each role has an expert who will guide the team members assigned to the role. 
4. Each team member may be assigned to more than one role at any given time. 
5. No team members, besides the experts, will be assigned to the same role in two consecutive cycles. 
6. Experts may be assigned to other roles just like any other team member. 

In [55]:
import pandas as pd
import numpy as np
import random
random.seed(42)

# Assign Roles 

This part of the code is used to assign roles to the team members from scratch

Instantiate list of members

In [56]:
members = [
            "Oluwaseyi Adaramola",
            "Farook Ajose",
            "Malik Akintola",
            "Ibukun Akinleye",
            "Samson Ilesanmi",
            "Oluwaseunfunmi Alarima",
            "Damilola Omokehinde",
            "Samuel Olayiwola",
            "Adejuwon Adegite",
            "Kafilat Akinyemi",
            "Chibike Enibe",
            "Ayodeji Abatan",
            "Divine Nkwocha",
            "AbdulQahhar Okenla",
        ]

members

['Oluwaseyi Adaramola',
 'Farook Ajose',
 'Malik Akintola',
 'Ibukun Akinleye',
 'Samson Ilesanmi',
 'Oluwaseunfunmi Alarima',
 'Damilola Omokehinde',
 'Samuel Olayiwola',
 'Adejuwon Adegite',
 'Kafilat Akinyemi',
 'Chibike Enibe',
 'Ayodeji Abatan',
 'Divine Nkwocha',
 'AbdulQahhar Okenla']

Instantiate available roles

In [57]:
roles = pd.DataFrame({
    "role": ["Project Manager",
            "Data Analyst (Python)",
            "Data Scientist/Statistician",
            "Visualization Specialist (Power BI)",
            "Technical Writer",
            "Publicity Coordinator",],
    "num_people": [2, 4, 4, 4, 4, 3],
    "expert": ["Farook Ajose",
               "Malik Akintola",
               "Ayodeji Abatan",
               "Samson Ilesanmi",
               "Farook Ajose",
               "AbdulQahhar Okenla"]
    })
roles

,role,num_people,expert
0,Project Manager,2,Farook Ajose
1,Data Analyst (Python),4,Malik Akintola
2,Data Scientist/Statistician,4,Ayodeji Abatan
3,Visualization Specialist (Power BI),4,Samson Ilesanmi
4,Technical Writer,4,Farook Ajose
5,Publicity Coordinator,3,AbdulQahhar Okenla


Write function to assign roles

In [58]:
num_cycles = 6

def assign_roles(row, cycles=num_cycles, team=members):
    """Assign team members to different roles for each rotation cycle, ensuring that an expert in the area is present to guide them. 

    Args:
        row (pandas.Series): A row of the `roles` dataframe containing the role details.
        cycles (int, optional): Number of rotation cycles to assign the roles for. Defaults to the value of `num_cycles`.
        team (list, optional): A list of team members to select from. Defaults to the value of `members`.

    Returns:
        expert_cycle_dict (dict): A dictionary containing the assigned members for each cycle, with keys in the format "Cycle x" where x is the cycle number.
    """
    
    # Remove expert from sample list 
    expert = row['expert']
    no_expert_team = team.copy()
    no_expert_team.remove(expert)

    # Create dictionary of assigned members for each cycle
    cycle_dict = {
        f"Cycle {cycle}":  np.random.choice(no_expert_team, size=row["num_people"]-1, replace=False)
        for cycle in range(1, cycles+1)
    }
    
    # Remove common values between consecutive cycles
    for i in range(1, cycles):
        current_cycle = cycle_dict[f"Cycle {i}"]
        next_cycle = cycle_dict[f"Cycle {i+1}"]
        common_members = np.intersect1d(current_cycle, next_cycle)
        for member in common_members:
            new_member = np.random.choice(no_expert_team, size=1)[0]
            while new_member in next_cycle:
                new_member = np.random.choice(no_expert_team, size=1)[0]
            next_cycle = np.where(next_cycle == member, new_member, next_cycle)
        cycle_dict[f"Cycle {i+1}"] = next_cycle
        
    # Append expert to list and convert to string    
    expert_cycle_dict = {
        f"Cycle {cycle}":  ", ".join(np.append(expert, cycle_dict[f"Cycle {cycle}"]))
        for cycle in range(1, cycles+1)
    }

    return expert_cycle_dict
        
# Test function to see outputs.     
assign_roles(roles.iloc[2, :])

{'Cycle 1': 'Ayodeji Abatan, Divine Nkwocha, Samson Ilesanmi, Oluwaseyi Adaramola',
 'Cycle 2': 'Ayodeji Abatan, Chibike Enibe, Adejuwon Adegite, AbdulQahhar Okenla',
 'Cycle 3': 'Ayodeji Abatan, Divine Nkwocha, Malik Akintola, Kafilat Akinyemi',
 'Cycle 4': 'Ayodeji Abatan, Oluwaseyi Adaramola, Ibukun Akinleye, Samson Ilesanmi',
 'Cycle 5': 'Ayodeji Abatan, Damilola Omokehinde, Kafilat Akinyemi, AbdulQahhar Okenla',
 'Cycle 6': 'Ayodeji Abatan, Samuel Olayiwola, Adejuwon Adegite, Chibike Enibe'}

Update dataframe with function

In [59]:
roles[[f"Cycle {cycle}" for cycle in range(1, num_cycles+1)]] = roles.apply(func=assign_roles, axis=1, result_type='expand')
roles

,role,num_people,expert,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
0,Project Manager,2,Farook Ajose,"Farook Ajose, Adejuwon Adegite","Farook Ajose, Malik Akintola","Farook Ajose, Samson Ilesanmi","Farook Ajose, Ayodeji Abatan","Farook Ajose, Adejuwon Adegite","Farook Ajose, Damilola Omokehinde"
1,Data Analyst (Python),4,Malik Akintola,"Malik Akintola, Oluwaseunfunmi Alarima, Oluwas...","Malik Akintola, Damilola Omokehinde, Samson Il...","Malik Akintola, Farook Ajose, AbdulQahhar Oken...","Malik Akintola, Adejuwon Adegite, Chibike Enib...","Malik Akintola, AbdulQahhar Okenla, Ibukun Aki...","Malik Akintola, Damilola Omokehinde, Samuel Ol..."
2,Data Scientist/Statistician,4,Ayodeji Abatan,"Ayodeji Abatan, Samson Ilesanmi, AbdulQahhar O...","Ayodeji Abatan, Adejuwon Adegite, Divine Nkwoc...","Ayodeji Abatan, Oluwaseunfunmi Alarima, Kafila...","Ayodeji Abatan, Farook Ajose, AbdulQahhar Oken...","Ayodeji Abatan, Malik Akintola, Oluwaseunfunmi...","Ayodeji Abatan, Samson Ilesanmi, Ibukun Akinle..."
3,Visualization Specialist (Power BI),4,Samson Ilesanmi,"Samson Ilesanmi, Chibike Enibe, Damilola Omoke...","Samson Ilesanmi, Oluwaseyi Adaramola, Divine N...","Samson Ilesanmi, AbdulQahhar Okenla, Farook Aj...","Samson Ilesanmi, Oluwaseyi Adaramola, Chibike ...","Samson Ilesanmi, Adejuwon Adegite, AbdulQahhar...","Samson Ilesanmi, Ibukun Akinleye, Farook Ajose..."
4,Technical Writer,4,Farook Ajose,"Farook Ajose, Damilola Omokehinde, Malik Akint...","Farook Ajose, Ibukun Akinleye, Oluwaseyi Adara...","Farook Ajose, AbdulQahhar Okenla, Divine Nkwoc...","Farook Ajose, Ibukun Akinleye, Chibike Enibe, ...","Farook Ajose, Samuel Olayiwola, Adejuwon Adegi...","Farook Ajose, Oluwaseyi Adaramola, Malik Akint..."
5,Publicity Coordinator,3,AbdulQahhar Okenla,"AbdulQahhar Okenla, Ayodeji Abatan, Kafilat Ak...","AbdulQahhar Okenla, Samuel Olayiwola, Samson I...","AbdulQahhar Okenla, Chibike Enibe, Divine Nkwocha","AbdulQahhar Okenla, Ibukun Akinleye, Malik Aki...","AbdulQahhar Okenla, Ayodeji Abatan, Samuel Ola...","AbdulQahhar Okenla, Damilola Omokehinde, Kafil..."


Summarize the roles for each member per cycle

In [60]:
def summarize_roles(df, members=members, cycles=num_cycles):
    role_summary = {}
    for member in members:
        member_roles = []
        for cycle in range(1, cycles+1):
            assigned_roles = ", ".join(df["role"][df[f"Cycle {cycle}"].str.contains(member)])
            member_roles.append(assigned_roles)
        role_summary[member] = member_roles
        
    df = pd.DataFrame(role_summary).T
    df.columns = [f"Cycle {cycle}" for cycle in range(1, cycles+1)]
    
    return df

In [61]:
summarize_roles(roles)

,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
Oluwaseyi Adaramola,"Data Analyst (Python), Data Scientist/Statisti...","Visualization Specialist (Power BI), Technical...",,Visualization Specialist (Power BI),,"Data Analyst (Python), Technical Writer"
Farook Ajose,"Project Manager, Data Analyst (Python), Techni...","Project Manager, Technical Writer","Project Manager, Data Analyst (Python), Visual...","Project Manager, Data Scientist/Statistician, ...","Project Manager, Data Analyst (Python), Techni...","Project Manager, Visualization Specialist (Pow..."
Malik Akintola,"Data Analyst (Python), Visualization Specialis...","Project Manager, Data Analyst (Python), Data S...","Data Analyst (Python), Visualization Specialis...","Data Analyst (Python), Technical Writer, Publi...","Data Analyst (Python), Data Scientist/Statisti...","Data Analyst (Python), Technical Writer"
Ibukun Akinleye,,Technical Writer,,"Technical Writer, Publicity Coordinator",Data Analyst (Python),"Data Scientist/Statistician, Visualization Spe..."
Samson Ilesanmi,"Data Scientist/Statistician, Visualization Spe...","Data Analyst (Python), Visualization Specialis...","Project Manager, Visualization Specialist (Pow...",Visualization Specialist (Power BI),Visualization Specialist (Power BI),"Data Scientist/Statistician, Visualization Spe..."
Oluwaseunfunmi Alarima,Data Analyst (Python),,"Data Scientist/Statistician, Technical Writer",,Data Scientist/Statistician,
Damilola Omokehinde,"Visualization Specialist (Power BI), Technical...",Data Analyst (Python),Data Scientist/Statistician,Visualization Specialist (Power BI),,"Project Manager, Data Analyst (Python), Public..."
Samuel Olayiwola,,"Visualization Specialist (Power BI), Technical...",,,"Technical Writer, Publicity Coordinator",Data Analyst (Python)
Adejuwon Adegite,Project Manager,Data Scientist/Statistician,,"Data Analyst (Python), Data Scientist/Statisti...","Project Manager, Visualization Specialist (Pow...",
Kafilat Akinyemi,Publicity Coordinator,,Data Scientist/Statistician,,"Data Scientist/Statistician, Visualization Spe...",Publicity Coordinator


Save dataframe to CSV

In [62]:
# roles.to_csv("assigned_roles.csv", index=False)

# Update Existing Roles 

This part of the code is used to update the existing roles in a CSV.

In [63]:
current_roles = pd.read_csv("assigned_roles.csv")
current_roles.head()

,role,num_people,expert,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
0,Data Analyst (Python),4,Malik Akintola,"Malik Akintola, Chibike Enibe, AbdulQahhar Oke...","Malik Akintola, Samson Ilesanmi, Kafilat Akiny...","Malik Akintola, Adejuwon Adegite, Chibike Enib...","Malik Akintola, AbdulQahhar Okenla, Farook Ajo...","Malik Akintola, Adejuwon Adegite, Ayodeji Abat...","Malik Akintola, Farook Ajose, Chibike Enibe, D..."
1,Data Scientist/Statistician,4,Ayodeji Abatan,"Ayodeji Abatan, AbdulQahhar Okenla, Divine Nkw...","Ayodeji Abatan, Kafilat Akinyemi, Chibike Enib...","Ayodeji Abatan, Ibukun Akinleye, Oluwaseyi Ada...","Ayodeji Abatan, Oluwaseunfunmi Alarima, Samson...","Ayodeji Abatan, Samuel Olayiwola, Farook Ajose...","Ayodeji Abatan, Malik Akintola, Oluwaseyi Adar..."
2,Visualization Specialist (Power BI),4,Samson Ilesanmi,"Samson Ilesanmi, Ayodeji Abatan, Damilola Omok...","Samson Ilesanmi, Ibukun Akinleye, Farook Ajose...","Samson Ilesanmi, Samuel Olayiwola, Divine Nkwo...","Samson Ilesanmi, Ayodeji Abatan, AbdulQahhar O...","Samson Ilesanmi, Ibukun Akinleye, Oluwaseunfun...","Samson Ilesanmi, AbdulQahhar Okenla, Divine Nk..."
3,Technical Writer,4,Farook Ajose,"Farook Ajose, Oluwaseunfunmi Alarima, AbdulQah...","Farook Ajose, Samson Ilesanmi, Adejuwon Adegit...","Farook Ajose, Chibike Enibe, Samuel Olayiwola,...","Farook Ajose, Oluwaseyi Adaramola, AbdulQahhar...","Farook Ajose, Ibukun Akinleye, Chibike Enibe, ...","Farook Ajose, AbdulQahhar Okenla, Oluwaseyi Ad..."
4,Publicity Coordinator,3,AbdulQahhar Okenla,"AbdulQahhar Okenla, Ibukun Akinleye, Samuel Ol...","AbdulQahhar Okenla, Damilola Omokehinde, Adeju...","AbdulQahhar Okenla, Malik Akintola, Ibukun Aki...","AbdulQahhar Okenla, Samson Ilesanmi, Adejuwon ...","AbdulQahhar Okenla, Ibukun Akinleye, Kafilat A...","AbdulQahhar Okenla, Oluwaseyi Adaramola, Divin..."


In [65]:
summarize_roles(current_roles)

,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
Oluwaseyi Adaramola,Data Analyst (Python),,"Data Scientist/Statistician, Visualization Spe...",Technical Writer,,"Data Scientist/Statistician, Technical Writer,..."
Farook Ajose,Technical Writer,"Data Scientist/Statistician, Visualization Spe...",Technical Writer,"Data Analyst (Python), Technical Writer","Data Scientist/Statistician, Technical Writer","Data Analyst (Python), Technical Writer"
Malik Akintola,"Data Analyst (Python), Data Scientist/Statisti...",Data Analyst (Python),"Data Analyst (Python), Publicity Coordinator","Data Analyst (Python), Visualization Specialis...",Data Analyst (Python),"Data Analyst (Python), Data Scientist/Statisti..."
Ibukun Akinleye,Publicity Coordinator,"Data Analyst (Python), Visualization Specialis...","Data Scientist/Statistician, Technical Writer,...",,"Visualization Specialist (Power BI), Technical...",
Samson Ilesanmi,Visualization Specialist (Power BI),"Data Analyst (Python), Visualization Specialis...",Visualization Specialist (Power BI),"Data Scientist/Statistician, Visualization Spe...","Visualization Specialist (Power BI), Technical...",Visualization Specialist (Power BI)
Oluwaseunfunmi Alarima,Technical Writer,,,Data Scientist/Statistician,Visualization Specialist (Power BI),
Damilola Omokehinde,Visualization Specialist (Power BI),Publicity Coordinator,Data Scientist/Statistician,Data Analyst (Python),Visualization Specialist (Power BI),Data Analyst (Python)
Samuel Olayiwola,Publicity Coordinator,,"Data Analyst (Python), Visualization Specialis...",,Data Scientist/Statistician,
Adejuwon Adegite,,"Technical Writer, Publicity Coordinator",Data Analyst (Python),Publicity Coordinator,Data Analyst (Python),
Kafilat Akinyemi,Visualization Specialist (Power BI),"Data Analyst (Python), Data Scientist/Statisti...",,,"Data Analyst (Python), Publicity Coordinator",Data Scientist/Statistician
